# Table of Contents
0. Hugging Face Login & Unzipping Dataset
1. Setup & Installs
2. Configuration & Utilities
3. Data Loading (Train / Validation / Test)
4. Train/Validation Split
5. Whisper Model Setup & Preprocessing
6. Create Datasets with Audio Feature 
7. Preprocessing Function
8. Apply Preprocessing (Memory-Safe)
9. Training (Seq2SeqTrainer)
 10. Evaluation 
 11. Saving the model

## Final Model Hugging Face Link:
https://huggingface.co/Rohan432/Augmented_on_normal

# Bangla Dialect ASR with Whisper (Optimized)

**Automatic Speech Recognition for Bangladeshi Regional Dialects**


~ **Whisper-small** fine-tuning for dialect ASR


## Key Features:
- Proper seeding for reproducibility
-  Optimized training with label smoothing, weight decay, gradient clipping
-  WER + Normalized Levenshtein Similarity metrics
-  Memory-efficient sequential processing
-  Colab T4 GPU compatible


## 0. Hugging Face Login & Unzipping Dataset

In [1]:
!unzip -q shobdotori -d /content/

unzip:  cannot find or open shobdotori, shobdotori.zip or shobdotori.ZIP.


In [ ]:
!pip -q install huggingface_hub transformers
from huggingface_hub import notebook_login

notebook_login()

## 1. Setup & Installs


In [1]:
# Install all required packages with compatible versions
import os
os.environ['PYTHONWARNINGS'] = 'ignore::FutureWarning,ignore::DeprecationWarning'

# Uninstalling  potentially conflicting packages first
!pip uninstall -y transformers accelerate -q

# Install core packages with compatible versions
!pip install -q transformers==4.44.2 accelerate==0.33.0
!pip install -q datasets==2.20.0 evaluate==0.4.1 jiwer==3.0.3
!pip install -q librosa==0.10.1 soundfile==0.12.1 tqdm
!pip install -q scikit-learn pandas


!pip install -q fsspec==2025.3.0


!pip install -q sentencepiece

print("\n" + "="*80)
print(" All dependencies installed successfully!")
print("="*80)

print("\n  restart runtime if this is the first installation.")

print("="*80)
import random
import numpy as np
import torch

def set_seed(seed: int = 42):
    """Set seed for reproducibility across all libraries"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    print(f" Global seed set to {seed}")

# Set seed globally
set_seed(42)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.5.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.20.0 requires fsspec[http]<=2024.5.0,>=2023.1.0, but you have fsspec 2025.3.0 which is incompatible.

 All dependencies installed successfully!

  restart runtime if this is the first installation.
 Global seed set to 42


###  IMPORTANT: Restart Runtime



In [ ]:
# Verify installations and check versions
import sys
import torch
import transformers
import datasets
import librosa
import soundfile
import numpy as np

print("="*60)
print("ENVIRONMENT VERIFICATION")
print("="*60)
print(f"Python version: {sys.version.split()[0]}")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Datasets version: {datasets.__version__}")
print(f"Librosa version: {librosa.__version__}")
print(f"Soundfile version: {soundfile.__version__}")
print(f"NumPy version: {np.__version__}")
print("\n" + "="*60)
print("CUDA INFO")
print("="*60)
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print("="*60)

ENVIRONMENT VERIFICATION
Python version: 3.12.12
PyTorch version: 2.8.0+cu126
Transformers version: 4.44.2
Datasets version: 2.20.0
Librosa version: 0.10.1
Soundfile version: 0.12.1
NumPy version: 1.26.4

CUDA INFO
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.83 GB


## 2. Configuration & Utilities

In [ ]:
# Mount Google Drive
# from google.colab import drive
# drive.mount("/content/drive")
# print("✓ Google Drive mounted successfully!")

In [ ]:
import os
from pathlib import Path

class Config:
    # Paths - UPDATE THESE IF YOUR DATA IS IN A DIFFERENT LOCATION
    BASE_PATH = "/content"
    TRAIN_AUDIO_PATH = f"{BASE_PATH}/Train"
    TRAIN_ANNOTATION_PATH = f"{BASE_PATH}/Train_annotation"
    TEST_AUDIO_PATH = f"{BASE_PATH}/Test"
    OUTPUT_DIR = "./whisper-bangla-dialect"


    # Whisper Model
    MODEL_NAME = "ishmamzarif/bangla_asr_augmented_bangla-whisper-epoch-11"
    LANGUAGE = "bengali"
    TASK = "transcribe"


    # Training parameters
    BATCH_SIZE = 4
    GRADIENT_ACCUMULATION_STEPS = 4  # Effective batch size = 16
    LEARNING_RATE = 2e-5
    WARMUP_STEPS = 400
    NUM_TRAIN_EPOCHS = 8
    LOGGING_STEPS = 50


    # Audio processing
    SAMPLING_RATE = 16000
    MAX_AUDIO_LENGTH = 30  # seconds



# Verify paths exist
print("Checking paths...")
print(f" Base path exists: {os.path.exists(Config.BASE_PATH)}")
print(f" Train audio exists: {os.path.exists(Config.TRAIN_AUDIO_PATH)}")
print(f" Train annotations exist: {os.path.exists(Config.TRAIN_ANNOTATION_PATH)}")
print(f" Test audio exists: {os.path.exists(Config.TEST_AUDIO_PATH)}")

if not os.path.exists(Config.TRAIN_AUDIO_PATH):
    print("\n WARNING: Update Config.BASE_PATH ")

Checking paths...
 Base path exists: True
 Train audio exists: True
 Train annotations exist: True
 Test audio exists: True


## 3. Data Loading (Train / Validation / Test)

In [ ]:
import pandas as pd

def load_training_data(train_audio_path, train_annotation_path):
    """Load and combine all regional dialect data"""
    all_data = []

    # Get all region folders
    region_folders = sorted([f for f in os.listdir(train_audio_path)
                            if os.path.isdir(os.path.join(train_audio_path, f))])

    print(f"Found {len(region_folders)} regional dialects")

    for region in region_folders:
        # Find corresponding CSV file (case-insensitive)
        csv_files = [f for f in os.listdir(train_annotation_path)
                    if region.lower() in f.lower() and f.endswith('.csv')]

        if not csv_files:
            print(f" Warning: No CSV found for region {region}")
            continue

        csv_path = os.path.join(train_annotation_path, csv_files[0])
        df = pd.read_csv(csv_path)

        # Add full audio paths
        audio_dir = os.path.join(train_audio_path, region)
        df['audio_path'] = df['audio'].apply(lambda x: os.path.join(audio_dir, x))
        df['region'] = region

        # Verify files exist
        before_count = len(df)
        df = df[df['audio_path'].apply(os.path.exists)]
        after_count = len(df)

        if before_count != after_count:
            print(f" {region}: {before_count - after_count} audio files not found")

        all_data.append(df)
        print(f"Loaded {len(df)} samples from {region}")

    # Combine all data
    combined_df = pd.concat(all_data, ignore_index=True)

    # RemovING any rows with empty text
    combined_df = combined_df[combined_df['text'].notna() & (combined_df['text'].str.strip() != '')]

    print(f"\n{'='*60}")
    print(f"Total training samples: {len(combined_df)}")
    print(f"{'='*60}")

    return combined_df

# Loading the data
print("Loading training data...\n")
train_df = load_training_data(Config.TRAIN_AUDIO_PATH, Config.TRAIN_ANNOTATION_PATH)

print("\nSample data:")
print(train_df[['audio', 'text', 'region']].head(3))

Loading training data...

Found 20 regional dialects
Loaded 39 samples from Barisal
Loaded 189 samples from Bhola
Loaded 114 samples from Bogura
Loaded 107 samples from Brahmanbaria
Loaded 401 samples from Chittagong
Loaded 53 samples from Comilla
Loaded 115 samples from Dhaka
Loaded 66 samples from Feni
Loaded 33 samples from Jessore
Loaded 270 samples from Jhenaidah
Loaded 21 samples from Khulna
Loaded 53 samples from Kushtia
Loaded 400 samples from Lakshmipur
Loaded 400 samples from Mymensingh
Loaded 108 samples from Natore
Loaded 61 samples from Noakhali
Loaded 166 samples from Pabna
Loaded 73 samples from Rajshahi
Loaded 357 samples from Rangpur
Loaded 324 samples from Sylhet

Total training samples: 3350

Sample data:
                  audio                             text   region
0  female_barisal_1.wav  আজ সকালে আমি বাজারে গিয়েছিলাম।  Barisal
1  female_barisal_3.wav            আকাশে আজ মেঘ জমে আছে।  Barisal
2  female_barisal_4.wav   আমি বন্ধুদের সাথে খেলা খেলেছি।  Barisal


## 4. Train/Validation Split

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting into train and validation (90/10 split, stratified by region)
train_split, val_split = train_test_split(
    train_df,
    test_size=0.1,
    random_state=42,
    stratify=train_df['region']
)

print(f"Training samples: {len(train_split)}")
print(f"Validation samples: {len(val_split)}")
print(f"\nValidation region distribution:")
print(val_split['region'].value_counts().sort_index())

Training samples: 3015
Validation samples: 335

Validation region distribution:
region
Barisal          4
Bhola           19
Bogura          11
Brahmanbaria    11
Chittagong      40
Comilla          5
Dhaka           12
Feni             7
Jessore          3
Jhenaidah       27
Khulna           2
Kushtia          5
Lakshmipur      40
Mymensingh      40
Natore          11
Noakhali         6
Pabna           17
Rajshahi         7
Rangpur         36
Sylhet          32
Name: count, dtype: int64


## 5. Whisper Model Setup & Preprocessing

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

print(f"Loading {Config.MODEL_NAME}...")

# Load processor with language and task
processor = WhisperProcessor.from_pretrained(
    Config.MODEL_NAME,
    language=Config.LANGUAGE,
    task=Config.TASK
)

# Load model
model = WhisperForConditionalGeneration.from_pretrained(Config.MODEL_NAME)

# Explicitly set pad_token_id for model and tokenizer
if model.config.pad_token_id is None:

    model.config.pad_token_id = processor.tokenizer.pad_token_id = processor.tokenizer.eos_token_id
    print(f"  Set model.config.pad_token_id and processor.tokenizer.pad_token_id to: {model.config.pad_token_id} (EOS token ID)")
elif processor.tokenizer.pad_token_id is None:
    processor.tokenizer.pad_token_id = model.config.pad_token_id
    print(f"  Set processor.tokenizer.pad_token_id to: {processor.tokenizer.pad_token_id} (from model config)")


# Configuring model for training with proper forced decoder IDs
forced_decoder_ids = processor.get_decoder_prompt_ids(
    language=Config.LANGUAGE,
    task=Config.TASK
)
model.config.forced_decoder_ids = forced_decoder_ids
model.config.suppress_tokens = None
model.config.use_cache = False

print("✓ Model and processor loaded successfully!")

# ------------------------------------------------------
# HF Repo Naming Utility (NEW REPO FOR EACH EPOCH)
# ------------------------------------------------------
def repo_name_for_epoch(base, epoch):
    return f"{base}-epoch-{epoch}"

from transformers import TrainerCallback

class PushEachEpochCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        epoch = int(state.epoch)
        repo = repo_name_for_epoch("bangla-whisper", epoch)
        print(f"\n📤 [Epoch {epoch}] Pushing model to HF repo: {repo}\n")

        model = kwargs["model"]
        model.push_to_hub(repo)

        try:
            processor.push_to_hub(repo)
        except:
            pass

        try:
            tokenizer.push_to_hub(repo)
        except:
            pass

        return control

print(f"  Model parameters: {model.num_parameters() / 1e6:.1f}M")
print(f"  Language: {Config.LANGUAGE}")
print(f"  Task: {Config.TASK}")

Loading ishmamzarif/bangla_asr_augmented_bangla-whisper-epoch-11...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

✓ Model and processor loaded successfully!
  Model parameters: 241.7M
  Language: bengali
  Task: transcribe


## 6. Create Datasets with Audio Feature

In [ ]:
from datasets import Dataset, Audio
import gc

def create_dataset(df, sampling_rate=16000):
    """Create HuggingFace Dataset with Audio feature"""
    dataset_dict = {
        'audio': df['audio_path'].tolist(),
        'text': df['text'].tolist(),
        'region': df['region'].tolist()
    }

    dataset = Dataset.from_dict(dataset_dict)

    # Cast audio column - this handles resampling to 16kHz automatically
    # Uses soundfile/librosa under the hood
    dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate, mono=True))

    return dataset

print("Creating datasets with Audio feature...")
train_dataset = create_dataset(train_split, Config.SAMPLING_RATE)
val_dataset = create_dataset(val_split, Config.SAMPLING_RATE)

print(f" Train dataset: {len(train_dataset)} samples")
print(f" Val dataset: {len(val_dataset)} samples")

# Clear memory
gc.collect()

Creating datasets with Audio feature...
 Train dataset: 3015 samples
 Val dataset: 335 samples


10346

## 7. Preprocessing Function

In [ ]:
def prepare_dataset(batch):
    """Preprocess audio and text for Whisper training"""
    # Load audio (already resampled to 16kHz by Audio feature)
    audio = batch["audio"]
    audio_array = audio["array"]

    # Truncate if too long (30 seconds max)
    max_samples = Config.MAX_AUDIO_LENGTH * Config.SAMPLING_RATE
    if len(audio_array) > max_samples:
        audio_array = audio_array[:max_samples]

    # Compute log-Mel spectrogram features
    batch["input_features"] = processor.feature_extractor(
        audio_array,
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    # Tokenize text labels
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids

    return batch

print("Preprocessing function defined!")

Preprocessing function defined!


## 8. Apply Preprocessing (Memory-Safe)

In [ ]:
print("Preprocessing datasets...")


# Process training data
print("Processing training data...")
train_dataset = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names,
    desc="Processing training data",
    num_proc=1,  # Sequential processing to avoid memory issues
)

# Force garbage collection
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("\nProcessing validation data...")
val_dataset = val_dataset.map(
    prepare_dataset,
    remove_columns=val_dataset.column_names,
    desc="Processing validation data",
    num_proc=1,
)

# Clear memory again
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f"\n Preprocessing complete!")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Val samples: {len(val_dataset)}")

Preprocessing datasets...
Processing training data...


Processing training data:   0%|          | 0/3015 [00:00<?, ? examples/s]


Processing validation data...


Processing validation data:   0%|          | 0/335 [00:00<?, ? examples/s]


 Preprocessing complete!
  Train samples: 3015
  Val samples: 335


## 9. Training (Seq2SeqTrainer)

### 9.1 Data Collator

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """Custom data collator for Whisper training with dynamic padding"""
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Get labels
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # If bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        # We explicitly create decoder_input ids by shifting labels


        # We Make a copy of labels & Replace -100  with the actual pad_token_id
        # because the decoder needs real tokens for its input, not -100.
        decoder_input_ids = labels.clone()
        decoder_input_ids[decoder_input_ids == -100] = self.processor.tokenizer.pad_token_id

        # Shift input_ids right: pad with self.decoder_start_token_id at the beginning.
        # This mirrors the model's internal `_shift_right` function.
        shifted_decoder_input_ids = decoder_input_ids.new_zeros(decoder_input_ids.shape)
        shifted_decoder_input_ids[:, 1:] = decoder_input_ids[:, :-1].clone()
        shifted_decoder_input_ids[:, 0] = self.decoder_start_token_id

        batch["decoder_input_ids"] = shifted_decoder_input_ids

        return batch

# Initialize data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id
)

print(" Data collator initialized!")

 Data collator initialized!


### 9.2 Evaluation Metrics (WER + Normalized Levenshtein Similarity)

In [ ]:
import evaluate
import numpy as np
from jiwer import cer

# Load WER metric
metric = evaluate.load("wer")

def normalized_levenshtein_similarity(refs, hyps):
    """Compute normalized Levenshtein similarity (1 - normalized edit distance)"""
    dist = cer(refs, hyps)  # cer returns normalized distance in [0, 1]
    return 1.0 - dist

def compute_metrics(pred):
    """Compute WER and Normalized Levenshtein Similarity metrics"""
    pred_ids = pred.predictions

    # Handle tuple predictions from generation
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]

    label_ids = pred.label_ids

    # Replace -100 with pad_token_id for proper decoding
    label_ids = np.where(label_ids != -100, label_ids, processor.tokenizer.pad_token_id)

    # Decode predictions and labels
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute metrics
    wer_val = metric.compute(predictions=pred_str, references=label_str)
    norm_lev_sim = normalized_levenshtein_similarity(label_str, pred_str)

    return {
        "wer": wer_val * 100.0,
        "norm_levenshtein_similarity": norm_lev_sim * 100.0,
    }



### 9.3 Training Configuration (Optimized)

In [ ]:
from transformers import Seq2SeqTrainingArguments
import gc

# Clear memory before training
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
use_fp16 = device == "cuda"

training_args = Seq2SeqTrainingArguments(
    output_dir=Config.OUTPUT_DIR,
    per_device_train_batch_size=Config.BATCH_SIZE,
    per_device_eval_batch_size=Config.BATCH_SIZE,
    gradient_accumulation_steps=Config.GRADIENT_ACCUMULATION_STEPS,
    learning_rate=Config.LEARNING_RATE,
    warmup_steps=Config.WARMUP_STEPS,
    num_train_epochs=Config.NUM_TRAIN_EPOCHS,
    gradient_checkpointing=True,
    fp16=use_fp16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=Config.LOGGING_STEPS,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="norm_levenshtein_similarity",  # Use similarity for best model
    greater_is_better=True,
    predict_with_generate=True,
    generation_max_length=225,
    save_total_limit=2,
    dataloader_num_workers=0,
    seed=42,  # Reproducibility
    # Optimization improvements
    lr_scheduler_type="cosine_with_restarts",
    weight_decay=0.01,
    label_smoothing_factor=0.1,
    max_grad_norm=1.0,
)

print(" Training arguments configured!")
print(f"  Device: {device}")
print(f"  Epochs: {Config.NUM_TRAIN_EPOCHS}")
print(f"  Batch size per device: {Config.BATCH_SIZE}")
print(f"  Gradient accumulation: {Config.GRADIENT_ACCUMULATION_STEPS}")
print(f"  Effective batch size: {Config.BATCH_SIZE * Config.GRADIENT_ACCUMULATION_STEPS}")
print(f"  Learning rate: {Config.LEARNING_RATE}")
print(f"  LR scheduler: cosine_with_restarts")
print(f"  Weight decay: 0.01")
print(f"  Label smoothing: 0.1")
print(f"  Max grad norm: 1.0")
print(f"  FP16 training: {use_fp16}")

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


 Training arguments configured!
  Device: cuda
  Epochs: 8
  Batch size per device: 4
  Gradient accumulation: 4
  Effective batch size: 16
  Learning rate: 2e-05
  LR scheduler: cosine_with_restarts
  Weight decay: 0.01
  Label smoothing: 0.1
  Max grad norm: 1.0
  FP16 training: True


### 9.4 Initialize Trainer

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.tokenizer,
    callbacks=[PushEachEpochCallback()],
)

print("✓ Trainer initialized!")
print(f"  Training samples: {len(train_dataset)}")
print(f"  Validation samples: {len(val_dataset)}")

/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


✓ Trainer initialized!
  Training samples: 3015
  Validation samples: 335


### 9.5 Start Training 

In [ ]:

if not torch.cuda.is_available():
    print("="*80)
    print(" WARNING: NO GPU DETECTED!")
    print("="*80)

    raise RuntimeError("GPU is necessary for training , otherwise it will take too much time")
else:
    print("="*80)
    print(" GPU DETECTED ")
    print("="*80)
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("="*80)

 GPU DETECTED 
GPU: Tesla T4
Memory: 15.8 GB


In [ ]:
import time

print("="*80)
print("Starting Whisper training...")
print("="*80)
print(f"Training on {device.upper()}")
print(f"Estimated time: ~2-4 hours on T4 GPU for {Config.NUM_TRAIN_EPOCHS} epochs\n")

start_time = time.time()

# Train the model
train_result = trainer.train()

end_time = time.time()
training_duration = end_time - start_time

print("\n" + "="*80)
print("✓ Whisper training completed!")
print(f"  Duration: {training_duration/3600:.2f} hours ({training_duration/60:.1f} minutes)")
print("="*80)

# Save training metrics
print("\nTraining metrics:")
for key, value in train_result.metrics.items():
    print(f"  {key}: {value}")

Starting Whisper training...
Training on CUDA
Estimated time: ~2-4 hours on T4 GPU for 8 epochs



/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Wer,Norm Levenshtein Similarity
0,2.347900,1.593980,26.519337,83.030303
2,1.538000,1.500336,18.011050,87.151515
4,1.466500,1.471246,12.486188,91.060606
6,1.444800,1.462361,10.607735,91.969697
7,1.441900,1.462102,10.441989,92.121212



📤 [Epoch 0] Pushing model to HF repo: bangla-whisper-epoch-0



Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}


...

New Data Upload               : |          |  0.00B /  0.00B            

  ...dqa06eh/model.safetensors:   0%|          |  881kB /  967MB            

README.md: 0.00B [00:00, ?B/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}
There were missing keys in the checkpoint model loaded: ['proj_out.weight'].



✓ Whisper training completed!
  Duration: 2.96 hours (177.8 minutes)

Training metrics:
  train_runtime: 10667.7333
  train_samples_per_second: 2.261
  train_steps_per_second: 0.141
  total_flos: 6.94249897549824e+18
  train_loss: 1.7192660487712699
  epoch: 7.978779840848806


## 10. Whisper Evaluation 

In [ ]:
print("Evaluating Whisper model on validation set...\n")

eval_results = trainer.evaluate()

print("Validation Results:")
print("="*60)
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")
print("="*60)

final_wer = eval_results.get('eval_wer', 0.0)
final_similarity = eval_results.get('eval_norm_levenshtein_similarity', 0.0)
print(f"\n✓ Final Validation WER: {final_wer:.2f}%")
print(f"✓ Final Normalized Levenshtein Similarity: {final_similarity:.2f}%")

Evaluating Whisper model on validation set...



Validation Results:
  eval_loss: 1.4621
  eval_wer: 10.4420
  eval_norm_levenshtein_similarity: 92.1212
  eval_runtime: 216.3039
  eval_samples_per_second: 1.5490
  eval_steps_per_second: 0.3880
  epoch: 7.9788

✓ Final Validation WER: 10.44%
✓ Final Normalized Levenshtein Similarity: 92.12%


## 11. Saving the model

In [ ]:
print("Saving final model...\n")

# Save model and processor
trainer.save_model(Config.OUTPUT_DIR)
processor.save_pretrained(Config.OUTPUT_DIR)

# ===================
# Pushing to HF Hub
from huggingface_hub import HfApi, HfFolder
from transformers import AutoModelForSequenceClassification, AutoTokenizer

repo_name = "Augmented_on_normal"
username = "Rohan432"
model_id = f"{username}/{repo_name}"

# Upload
from huggingface_hub import create_repo, upload_folder

create_repo(model_id, exist_ok=True)
upload_folder(
    folder_path=Config.OUTPUT_DIR,
    repo_id=model_id,
    path_in_repo=".",
)
# ====================

print(f"✓ Model saved to: {Config.OUTPUT_DIR}")
print(f"\nSaved files:")
for f in os.listdir(Config.OUTPUT_DIR):
    fpath = os.path.join(Config.OUTPUT_DIR, f)
    if os.path.isfile(fpath):
        size_mb = os.path.getsize(fpath) / (1024 * 1024)
        print(f"  {f}: {size_mb:.1f} MB")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}


Saving final model...



Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...nt-1319/model.safetensors:   2%|1         | 16.8MB /  967MB            

...

  ...ckpoint-1319/scheduler.pt:   8%|8         |   124B / 1.47kB            

  ...nt-1319/training_args.bin:   9%|8         |   497B / 5.84kB            

  ...kpoint-1504/rng_state.pth:   9%|8         | 1.25kB / 14.6kB            

  ...ckpoint-1504/scheduler.pt:   8%|8         |   124B / 1.47kB            

✓ Model saved to: ./whisper-bangla-dialect

Saved files:
  generation_config.json: 0.0 MB
  merges.txt: 0.5 MB
  normalizer.json: 0.1 MB
  added_tokens.json: 0.0 MB
  tokenizer_config.json: 0.0 MB
  preprocessor_config.json: 0.0 MB
  special_tokens_map.json: 0.0 MB
  config.json: 0.0 MB
  vocab.json: 1.0 MB
  training_args.bin: 0.0 MB
  model.safetensors: 922.2 MB
